In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import Planck13 as cosmo
import astropy.units as u
import astropy.constants as const
from scipy.optimize import fsolve
from scipy.interpolate import CubicSpline
from astroquery.sdss import SDSS
from astropy.table import vstack
import seaborn as sns
sns.set_context("paper",font_scale=1)
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"\usepackage{amsmath}\usepackage{mathptmx}",  # Times Roman
    "hatch.linewidth": 3.0,
})

def make_desig(data, ra_key='ra', dec_key='dec'):
    """make designation if df has 'ra' and 'dec' columns"""
    desig=[]
    for posstring in SkyCoord(data[ra_key].values*u.deg, data[dec_key].values*u.deg,frame='fk5').to_string("hmsdms"):
        posstring = posstring.split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        desig.append('J'+des_ra+des_dec)
    return desig


### look at some SNAP targets with neighboring sources

In [ ]:
snap = pd.read_csv("/home/insepien/research-data/alpaka/snap/snap146.csv")

# manually put in target number and approx angular sep
tarnum = np.array([37,42,45,56,58,91,105,112,142])-1
zs = np.array([snap.loc[i,"Z"] for i in tarnum])
sep_arcsec = np.array([15,15,10,15,10,15,15,15,10])
sep_kpc = (sep_arcsec*u.arcsec.to(u.rad)*cosmo.angular_diameter_distance(zs)).to(u.kpc)
snap.loc[tarnum]

## save cutouts from decals
# from urllib.request import urlretrieve
# for i in range(0, len(final_table)):
#urlretrieve('http://legacysurvey.org/viewer/jpeg-cutout?
#           ra='+str(snap.loc[tarnum[i],'ra_icrs'])+
#           '&dec='+str(snap.loc[tarnum[i],'dec_icrs'])+
#           '&layer=decals-dr7&pixscale=0.27&bands=grz',
#           'test.jpg'))

# decal links given coords, not sure what frame decals is but must not be off by a lot
# for i in range(len(tarnum)):
#     print(tarnum[i], "https://www.legacysurvey.org/viewer?ra="+str(snap.loc[tarnum[i],'ra_icrs'])+"&dec="+str(snap.loc[tarnum[i],'dec_icrs'])+"&layer=ls-dr9&zoom=15 ")
      
sep_kpc

### get magellan (39objs) & HST (171 from Lbol=OIII*800>1e46) wise lum and r-band mag
this was from the old 171 sample. We found later that OIII_DERRED from Mullaney is not reliable, since J1010 is brighter than all 171 objs in r-band and wise, but dimmer in OIII_DERRED. So OIII and IR not matching is probably not a problem

In [ ]:
def hst171():
    ################# magellan
    # this file is from query of sources on WISE, instructions for query in kris candidate selection notebook
    # wise search has 39 objects fitted in magellan sample
    wise39 = pd.read_pickle("/home/insepien/research-data/alpaka/wise-cat/wise_39fits.pkl")
    # read alpaka for optical comparison and redshift
    # alpaka has 41 rows, since J0926+0724 (1 is non-agn) and J1222-0007 (dual) are duplicated
    alpaka39 = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_39fits.pkl")
    alpaka39.reset_index(inplace=True)
    # get magellan mag and lum
    wise_magel_mag, wise_magel_lum, magel_irbol = get_wise_ir_lums(wise39,alpaka39)
    #################### OIII
    oiii = alpaka39['OIII_5007_LUM_DERRED']

    ################# HST
    # read wise search
    wise171 = pd.read_csv("/home/insepien/research-data/alpaka/wise-cat/wise_171.csv")
    desig = [wise171['designation'][i][:5]+wise171['designation'][i][10:15] for i in range(len(wise171))]
    wise171['DESIG'] = desig
    # find duplicated rows
    dupmask = wise171['DESIG'].duplicated(keep=False)
    # make separate df
    cols = ['DESIG',"ra","dec","ra_01","dec_01"]
    dups = wise171[dupmask][cols].copy()
    # add cols of ra and dec differences 
    dups['del_ra'] = (dups['ra']-dups['ra_01']).abs()
    dups['del_dec'] = (dups['dec']-dups['dec_01']).abs()
    # group df by name and find row index with lowest del_ra and del_dec and check
    min_dels = dups.groupby(by='DESIG').idxmin()
    if np.sum(min_dels['del_ra']==min_dels['del_dec'])==0:
        keep_ind = min_dels['del_ra'].values
    else:
        print("min del_ra does not match min del_dec")
    # turns out it is ok to do this
    keep_ind = min_dels['del_ra'].values
    # drop duplicated rows except for row with lowest ra and dec difference
    drop_ind = dups.index[~dups.index.isin(keep_ind)]
    wise171.drop(drop_ind,inplace=True)
    wise171.reset_index(inplace=True)
    #get luminosities
    hst_wise_mags, hst_wise_lums, hst_irbol = get_wise_ir_lums(wise171,mul171,wise_key='DESIG',mul_key='DESIG',wl_=22)
    # get OIII lum
    inf_mask = (mul171['OIII_5007_LUM_DERRED']>-np.inf) & (mul171['OIII_5007_LUM_DERRED']<np.inf)
    oiii171 = mul171['OIII_5007_LUM_DERRED'][inf_mask]
    # j1010 numbers
    j10_ipac['DESIG'] = make_desig(j10_ipac)
    j10_w_mag,j10_w_lum,j10_ir_bol = get_wise_ir_lums(j10_ipac,j10_mul,wise_key="DESIG",wl_=22)

    ################# r-band
    j10_r_magab = 16.83
    # read query results and calculate ab mag
    sdss = pd.read_pickle("~/research-data/alpaka/sdss-cat/sdss_rband_171.pkl")
    hst_magab = 22.5 - 2.5 * np.log10(sdss['spectroFlux_r'])

    fig,ax = plt.subplots(2,2,figsize=(13,10))
    ax[0,0].hist(wise171['w4mpro']+6.620,label='hst sample IR',color='green')
    ax[0,0].hist(pd.DataFrame(wise_magel_mag).loc[3]+6.620, label='magellan sample IR', color='orange') 
    ax[0,0].axvline(j10_ipac['w4mpro'].values[0]+6.620, label='J1010 IR',c='blue')
    ax[0,0].hist(hst_magab,label='hst sample SDSS-R',color='red')
    ax[0,0].axvline(j10_r_magab,c='red',label='j1010 SDSS-R')
    ax[0,0].set_title("Magnitude in WISE IR and SDSS R-band")
    ax[0,0].legend()
    ax[0,0].set_xlabel("AB Mag")

    # ax[0,1].hist(np.log10(hst_wise_lums),label='hst IR',color='green')
    # ax[0,1].hist(np.log10(wise_magel_lum),label='magellan IR',color='orange')
    ax[0,1].hist(np.log10(mul171['OIII_5007_LUM']+mul171['OIII_5007B_LUM']),label='narrow+broad [OIII] luminosity')
    ax[0,1].axvline(np.log10(j10_mul['OIII_5007_LUM']+j10_mul['OIII_5007B_LUM']).values[0],label='J1010 n+b [OIII] luminosity')
    ax[0,1].hist(np.log10(oiii171),label='OIII_DERRED',color='red')
    ax[0,1].axvline(np.log10(j10_mul['OIII_5007_LUM_DERRED'].values[0]),color='r',linestyle='--',label='j1010 OIII_DERRED')
    ax[0,1].set_xlabel("Log(Luminosity) [ergs/s]")
    ax[0,1].set_title("Compare [OIII] and [OIII] de-reddened luminosities")
    ax[0,1].legend()
    [ax[0,i].set_xlabel("Number of AGNs") for i in range(2)]

    ax[1,0].hist(np.log10(hst_wise_lums),label='hst sample',color='green')
    ax[1,0].hist(np.log10(wise_magel_lum),label='magellan sample',color='orange')
    ax[1,0].axvline(np.log10(j10_w_lum),label='j1010 IR',color='blue')
    ax[1,0].set_xlabel("Log(IR Luminosity) [ergs/s]")
    ax[1,0].set_title("WISE 22-micron Luminosity")
    ax[1,0].legend(loc='upper left')

    ax[1,1].hist(np.log10(hst_irbol),label='HST IR', color='green')
    ax[1,1].hist(np.log10(magel_irbol),label='Magellan IR', color='orange')
    ax[1,1].axvline(np.log10(j10_ir_bol),label='j1010 IR',c="blue")

    ax[1,1].set_xlabel("Log(L_bol)")
    ax[1,1].set_title("Bolometric luminosity (12% correction on IR lum)")
    ax[1,1].legend(loc='upper left')
    framenum=np.arange(1,5)
    axx = ax.ravel()
    [axx[i].text(0.9, 0.9, f"{framenum[i]}", transform=axx[i].transAxes, fontsize=30, 
                fontweight="bold", va="top", ha="left") for i in range(4)]
    fig.tight_layout();


### new sample
after knowing that OIII_DERRED is not reliable, we cut with wise IR lum, correct to bolometric by 12% in log(L_IR). Note that the IR luminosity calculated is a bit higher than the plot in J1010 proposal, but checked so many times with WISE doc (consistent to flux_dens) and chat GPT, and still don't find anything wrong.

first look at all type-2 agns in z=0.1-0.5 to see how low we can go in L

In [ ]:
mdf = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_z05_merged_wise.pkl")
j10_ipac = pd.read_csv("/home/insepien/research-data/alpaka/wise-cat/j1010_ipac.csv")
j10_mul = mdf[mdf['Desig']=="J1010+1413"]

wbol = mdf['irbol']
wmag = mdf['w4mpro']

if we cut around 46, we can get an ok sample, J1010 is still the brightest, so it might be unlikely that we find duals in our sample tho...? also 2nd plot is the plot in j1010 proposal, logL seems too be > by 0.3

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,3))

ax[0].hist(mdf['w4mpro']+6.620,histtype='step')
ax[0].axvline(j10_ipac['w4mpro'].values[0]+6.620,label='j1010')

a0= ax[1].hist(np.log10(wbol[(wbol<np.inf)&(wbol>-np.inf)]),histtype='step')
#ax[1].axvline(np.log10(j10_ir_bol),label='j1010')

ax[0].set_xlabel("wise AB mag at $22\mu m$")
ax[1].set_xlabel("Log(Lbol) [erg/s]")
ax[0].set_title("WISE magnitude of type-2 AGN in z=0.1-0.5")
ax[1].set_title("Bolometric luminosity\n(12% correction to $Log(L_{22 \mu m})$) ")

[ax[i].legend(loc='upper right') for i in range(2)];


fig, ax = plt.subplots(1,2,figsize=(8,3))
ax[0].scatter(np.log10(mdf['OIII_5007_LUM']),np.log10(mdf['wiseLum']),s=0.5)

ax[0].axvline(41,alpha=0.5)
ax[0].axhline(44,alpha=0.5)
ax[0].set_xlabel("Log(L$_{\\rm{[OIII]}})$ [erg/s]")

ax[1].scatter(np.log10(mdf['OIII_5007_LUM_DERRED']),np.log10(mdf['wiseLum']),s=0.5)
ax[1].set_xlabel("Log(L$_{\\rm{[OIII, derred]}})$ [erg/s]")

[ax[i].set_ylabel("Log(L$_{\\rm{IR}})$ [erg/s]") for i in range(2)]
[ax[i].set_xlim(xmin=40) for i in range(2)]
[ax[i].set_ylim(ymax=46) for i in range(2)]
fig.tight_layout()
;


now convert Lbol and Mbh for some eddington rate to choose a good L cut

In [ ]:
def lbol_to_m(lbol,edd_rate=0.3):
    ledd = lbol/edd_rate
    return np.log10(ledd/(1.28e46/1e8))

def m_to_lbol(m,edd_rate=0.3):
    ledd = 1.28e46*m/1e8
    return np.log10(ledd*edd_rate)

lbol_to_m(10**45.9)

look at snap sample at some lbol cut

In [ ]:
log_lcut = 45.9
# check that the log(lbol)>46 subset makes sense in OIII
w_Lcut = np.log10(wbol)>log_lcut
mul_Lcut = mdf[w_Lcut]
def get_lum(cat, key):
    return cat[key][(cat[key]>0) & (np.isfinite(cat[key]))]

fig,ax = plt.subplots(1,2,figsize=(8,3))
ax[0].hist(np.log10(get_lum(mul_Lcut,'HA_LUM')),label='HA',histtype='step')
ax[0].hist(np.log10(get_lum(mul_Lcut,'OIII_5007_LUM')),label='OIII',histtype='step')
ax[0].axvline(np.log10(j10_mul['HA_LUM'].values[0]),label='j1010 HA')
ax[0].axvline(np.log10(j10_mul['OIII_5007_LUM'].values[0]),label='j1010 OIII',c='orange')
ax[0].set_xlabel("Luminosity [erg/s]")
ax[0].legend()

ax[1].hist(mul_Lcut['Z'])
ax[1].set_xlabel("Z")
ax[1].set_ylabel("Number of AGN")
fig.suptitle(f"log(Lbol)={log_lcut}, log(MBH)={lbol_to_m(10**log_lcut):.2f},sample size: {len(mul_Lcut)}, {len(mul_Lcut)*0.1285/3:.0f} detections")
fig.tight_layout();

### get sdss r-band mag
we need to search for r-band mag for exposure time cal and also to put in APT file, but the query is messy, so I had to looked at duplicates and see what to remove/keep. Turns out there are actually 2 kpc scale duals, which messed things up for a while, but fixed everything. final sample after removing 4 observed w hst is 148. While preparing phase 2, I find that the dimmer J1222 is one of the target and it is below the luminosity cut. It was included by mistake because I selected things using designations.

In [ ]:
def search_sdss(cat,rad=1,query_field=['ra','dec','z','spectroFlux_r','spectroFluxIvar_r']):
    """query sdss specobj with ra and dec given some catalog"""
    search_results = []
    # search some radius in SDSS
    for i in cat.index.values:
        pos = SkyCoord(cat['RA'][i]*u.deg, cat['DEC'][i]*u.deg, frame='fk5')
        xid = SDSS.query_region(pos, radius=str(rad)+' arcsec',specobj_fields=query_field)
        search_results.append(xid)
    return search_results

def add_sdss_data(mul_Lcut):
    """open sdss query result df, clean that df, merge with alpaka, calculate r-mag AB and error"""
    ########### query around 2 arcsec. if around 1'', miss 1 obj
    #search_results = search_sdss(mul_Lcut)
    #sdss_matches = pd.concat([Table(search_results[i]).to_pandas() for i in range(len(search_results))])

    ########### put search results into df and remove dups
    sdss_matches = pd.read_pickle("/home/insepien/research-data/alpaka/snap/sdss_query_1arcsec_snap151_original.pkl")
    sdss_matches.reset_index(inplace=True,drop=True)
    sdss_matches['Desig'] = make_desig(sdss_matches)
    print(f"sdss query returns {sdss_matches.shape}")
    # match search results with alpaka ra and dec
    radec_mask = ((sdss_matches['ra'].isin(mul_Lcut['RA']))& (sdss_matches['dec'].isin(mul_Lcut['DEC'])))
    sdss_radec_match = sdss_matches[radec_mask]
    sdss_radec_match.reset_index(inplace=True,drop=True)
    print(f'matched ra&dec shape: {sdss_radec_match.shape}')
    # match ra and dec still return 155 rows. turns out there are duplicated sources (in RA, DEC) but slightly different z, 
    # since rmag is not too different, just keep the first row
    dup_radec_mask = (sdss_radec_match['ra'].duplicated(keep='first')) & (sdss_radec_match['dec'].duplicated(keep='first'))
    drop_ind = sdss_radec_match[dup_radec_mask].index.values
    sdss_nodups = sdss_radec_match.drop(drop_ind)
    sdss_nodups.reset_index(inplace=True,drop=True)
    print(f"removed dups shape: {sdss_nodups.shape}")

    ########### check what rows are missing
    missing_rows = mul_Lcut[~(mul_Lcut['Desig'].isin(sdss_nodups['Desig']))].reset_index(drop=True)
    print(f"number of missing rows: {len(missing_rows)}; index = {missing_rows.index.values}; names {missing_rows.Desig.values}")
    # add missing rows. this changes depending on what row is missing and why
    # 0th missing row due to no entry in specobj, 1st row missing due to too small search radius
    missing_ind = missing_rows.index.values
    query_specphot = lambda coord: SDSS.query_region(coord,radius='0.5 arcsec',fields=['ra','dec','z','modelMag_r','modelMagErr_r'])
    query_specobj = lambda coord: SDSS.query_region(coord,radius='2 arcsec',specobj_fields=['ra','dec','z','spectroFlux_r','spectroFluxIvar_r'])
    pos = lambda row: SkyCoord(row.RA*u.deg,row.DEC*u.deg,frame='fk5')
    search0 = query_specphot(pos(missing_rows.loc[0]))
    search1 = query_specobj(pos(missing_rows.loc[1]))
    # add rows to sdss df
    makerow = lambda row: list(row[['OBJID',"RA","DEC","Z"]].values)+[np.nan,np.nan,row['Desig']]
    sdss_nodups.loc[len(sdss_nodups)] = makerow(missing_rows.loc[0])
    sdss_nodups.loc[len(sdss_nodups)] = makerow(missing_rows.loc[1])
    # add 1st missing row fluxes
    closer_z_ind = np.argmin(search1['z']-missing_rows.loc[1,"Z"])
    ind0 = sdss_nodups[sdss_nodups['Desig'] == "J1203+2006"].index
    sdss_nodups.loc[ind0,'spectroFlux_r'] = search1['spectroFlux_r'].value[closer_z_ind]
    sdss_nodups.loc[ind0,'spectroFluxIvar_r'] = search1['spectroFluxIvar_r'].value[closer_z_ind]
    print(f"shape after adding missing rows: {sdss_nodups.shape}")
    # renaming colume to merge with alpaka
    sdss_nodups.rename(columns={'ra':"RA"},inplace=True)
    hst_sdss152 = mul_Lcut.merge(sdss_nodups[['RA', 'z', 'spectroFlux_r','spectroFluxIvar_r']],on='RA')
    # cal mag and mag err
    from uncertainties import ufloat
    import uncertainties.umath as um 
    magWErr = [22.5 - 2.5 * um.log10(ufloat(flx,err)) for flx,err in zip(hst_sdss152['spectroFlux_r'].values,1/np.sqrt(hst_sdss152['spectroFluxIvar_r'].values))]
    mag = [m.n for m in magWErr]
    magErr = [m.s for m in magWErr]
    hst_sdss152['rmag'] = mag
    hst_sdss152['rmagErr'] = magErr
    # add 0th missing row values which only has mag, not flux
    asinh_to_flx = lambda magr_asinh: um.sinh(magr_asinh/-2.5*um.log(10)-um.log(1.2e-10))*2.4e-10
    flx_to_ab = lambda flxd: -2.5*um.log10(flxd) 
    mag0933 = flx_to_ab(asinh_to_flx(ufloat(search0['modelMag_r'].value[0],search0['modelMagErr_r'].value[0])))
    ind1 = hst_sdss152[hst_sdss152['Desig'] == "J0933+2253"].index
    hst_sdss152.loc[ind1,'rmag'] = mag0933.n
    hst_sdss152.loc[ind1,'rmagErr'] = mag0933.s
    return hst_sdss152

#hst_sdss152 = add_sdss_data(mul_Lcut)

### exposure time stuffs

now that we have a sampe of 147 (148 in phase 1), I will just read in the pickled sample and I'll format stuffs here to help submit phase 2 

In [ ]:
hst_sdss152 = pd.read_pickle("/home/insepien/research-data/alpaka/snap/hstP2_snap151_alpakaWithSDSS.pkl")

hst_sdss152['10kpc_to_arcsec'] = ((10*u.kpc/cosmo.angular_diameter_distance(hst_sdss152['Z'].values)).to("")*u.rad).to(u.arcsec).value
hst_sdss152['oiii_flx_10'] = hst_sdss152['OIII_5007_FLUX']*1e-17 / np.pi/ hst_sdss152['10kpc_to_arcsec']**2  #erg/s/cm2/arcsec2
hst_sdss152['magEXT_10'] = hst_sdss152['rmag']+2.5*np.log10(np.pi*hst_sdss152['10kpc_to_arcsec'].values**2) #mag/arcsec2
hst_sdss152['rflx_Jy_10'] = hst_sdss152['spectroFlux_r']*3.631*u.Jy / np.pi/ hst_sdss152['10kpc_to_arcsec']**2 #Jy/arcsec2
hst_sdss152['OIII_wl'] = (hst_sdss152['Z']+1)*5007
hst_sdss152['HA_wl'] = (hst_sdss152['Z']+1)*6562
hst_sdss152['ha_flx_10'] = hst_sdss152['HA_FLUX']*1e-17 / np.pi/ hst_sdss152['10kpc_to_arcsec']**2  #erg/s/cm2/arcsec2

j1000mask = hst_sdss152['Desig']=='J1000+1242'
j1010mask = hst_sdss152['Desig']=='J1010+1413'

# remove 4 observed targets
observed_mask = hst_sdss152['Desig'].isin(['J1000+1242', 'J1010+1413','J1352+6541', 'J1356+1026','J1222-0007'])
hst_sdss148 = hst_sdss152[~observed_mask]
hst_sdss148.reset_index(inplace=True,drop=True)
print(f"final sample shape: {hst_sdss148.shape}")

hst_sdss152[j1000mask]

In [ ]:
zranges = [0.1,0.15,0.2,0.3,0.4]
zcuts=[(hst_sdss148['Z']< zranges[i+1]) & (hst_sdss148['Z'] > zranges[i]) for i in range(len(zranges)-1)]
max_cflx = []
colnames = ['Desig','Z','rmag','10kpc_to_arcsec','oiii_flx_10','magEXT_10','OIII_wl',"HA_wl",'ha_flx_10']
for cutind in range(4):
    maxpos = np.argmax(hst_sdss148['magEXT_10'][zcuts[cutind]])
    maxind = hst_sdss148[zcuts[cutind]].index.values[maxpos]
    max_cflx.append(hst_sdss148.loc[maxind:maxind][colnames].values[0])
max_cflx.append(hst_sdss152[j1000mask][colnames].values[0])
pd.DataFrame(max_cflx,columns=colnames)

### make the .csv for target import in APT

In [ ]:
# first get relevant columns and 
snap148 = hst_sdss148#[['Desig','RA','DEC',"Z",'wise4_mag','wise_Lbol','spectroFlux_r',
                      # 'rmag','rmagErr','OIII_5007_FLUX','10kpc_to_arcsec','oiii_flx_extended']]
snap148.reset_index(inplace=True,drop=True)

# convert coords to ICRS frame
coord_icrs = SkyCoord(snap148["RA"].values*u.deg, snap148["DEC"].values*u.deg,frame='fk5').transform_to("icrs")
snap148['ra_icrs'] = coord_icrs.ra.value
snap148['dec_icrs'] = coord_icrs.dec.value

#add unique name for APT entry. formatted as SDSSJ... with 2 decimals
dess = []
for i in range(len(snap148)):
    posstring = SkyCoord(snap148.loc[i,"ra_icrs"]*u.deg, snap148.loc[i,"dec_icrs"]*u.deg).to_string('hmsdms').split(" ")
    des_ra = posstring[0][0:2]+posstring[0][3:5]+posstring[0][6:11]
    des_dec = posstring[1][0:3]+posstring[1][4:6]+posstring[1][7:12]
    dess.append('SDSSJ'+des_ra+des_dec)
snap148['Name'] = dess
#add 'other flux' values
snap148['otherflux'] = [f"SDSS r-band AB mag = {snap148.loc[i,'rmag']:.3f}+/-{snap148.loc[i,'rmagErr']:.3f}" for i in range(len(snap148))]
#format csv file to import target
snap = snap148[["Name","ra_icrs","dec_icrs","Z","otherflux"]]
snap['category'] = ["GALAXY"]*len(snap)
snap['description'] = ["[NUCLEUS, QUASAR]"]*len(snap)
snap.shape
# # #snap.to_csv("snap_phase2.csv")

# fig,ax = plt.subplots(1,4,figsize=(10,3),dpi=200)
# ax[0].scatter(snap148['ra_icrs'],snap148['dec_icrs'],s=1)
# ax[0].set_xlim(0,360)
# ax[0].set_ylim(-90,90)
# ax[0].set_xlabel("RA[deg]")
# ax[0].set_ylabel("DEC[deg]")

# ax[1].hist(snap148['Z'])
# ax[1].set_xlabel("Redshift")

# ax[2].hist(snap148['rmag'])
# ax[2].axvline(hst_sdss152[j1010mask]['rmag'].values[0],c='g')
# ax[2].set_xlabel("R-mag AB [mag]")

# ax[3].hist(snap148['magEXT_10'])
# ax[3].axvline(hst_sdss152[j1010mask]['magEXT_10'].values[0],c='g',label='J1000')
# ax[3].set_xlabel("R-band surface brightness\n[mag/arcsec$^2$]")
# ax[3].legend(fontsize=6)

# fig.suptitle(f"Final SNAP sample, {len(snap148)} targets")
# fig.tight_layout()


comparing sample to J1000, alpaka oiii flux seems to be an underestimate

In [ ]:
def p(ax,quant,lab):
    ax.hist(quant,alpha=0.5)
    ax.axvline(quant[j1000mask].values[0],c='steelblue',label='j1000 sdss/alpaka')
    ax.set_xlabel(lab)

fig,ax = plt.subplots(1,2, figsize=(10,3))
p(ax[0],hst_sdss152['10kpc_to_arcsec'], "Extended source radius[arcsec]\n(assume phys. rad. of 10 kpc)")

p(ax[1],np.log10(hst_sdss152['oiii_flx_extended']), "Log(OIII_flux)\n[erg/s/cm2/arcsec2]")
ax[1].axvline(np.log10(1.7e-15),label='j1000 Magel',c='r')
ax[1].legend()


### choosing filter
here i plot the shifted OIII5007 and HA lines for the sample and the response function to choose filter for each observation. the idea is we want to observe NLR, i.e where there is OIII or HA emission. Then we want to observe just the continuum, i.e where there is no emission lines. So you need to choose on-band filter with the shifted emission line, and off-band filter without it.

In [ ]:
# info of WFC3 medium filters: central wl, width, max throughput
flt = {'547M':[5447.5,	650	,0.27],
        '621M':[6218.9,609.5,0.29],
       '689M':[6876.8,684.2,0.25],
       '763M':[7614.4,708.6,0.21],
       '845M':[8439.1,794.3,0.14]}

def plt_filter(fname,ax):
    """plot filter bandpasses"""
    tb = Table(fits.getdata("/home/insepien/research-data/alpaka/snap/uvis/wfc3_uvis2_f"+fname.lower()+".fits"))
    ax.plot(tb['WAVELENGTH'],tb['THROUGHPUT']*50,label='F'+fname,alpha=0.5)

def plt_subsample(cat,ax,zmin, zmax,color_,fillo3):
    """plot histogram of a subsample in some z-range
        option to fill oiii or ha just to emphasize which line to observe in that z-range subsample"""
    # cut by z range and redshift factor
    zmask = (cat['Z']< zmax) & (cat['Z'] > zmin)
    shift = 1+cat['Z'][zmask]
    # option to fill hist or not
    if fillo3:
        o3style='stepfilled'
        hastyle='step'
    else:
        o3style='step'
        hastyle='stepfilled'
    # plot shifted lines
    ax.hist(shift*5007,histtype=o3style,alpha=0.5,color=color_,label=f"z={zmin}-{zmax}")
    ax.hist(shift*6562,histtype=hastyle,alpha=0.5,color=color_,linestyle="--")


def onBand_mask(wl_line,flt_name,cat):
    """mask for on band for some filter with some central wl and width"""
    wl_eff,width,_ = flt[flt_name]
    wl_min = wl_eff - width/2
    wl_max = wl_eff + width/2
    return ((1+cat['Z'])*wl_line>wl_min) & ((1+cat['Z'])*wl_line < wl_max)

def offBand_mask(wl_line,flt_name,cat):
    """mask for on band for some filter with some central wl and width"""
    wl_eff,width,_ = flt[flt_name]
    wl_min = wl_eff - width/2
    wl_max = wl_eff + width/2
    return ((1+cat['Z'])*wl_line<wl_min) | ((1+cat['Z'])*wl_line > wl_max)

def test_filter_set(fname_on,fname_off,onbandline,cat):
    """test if some line onbandline is off filter fname_off and on fname_on"""
    d = {'OIII':5007, 'HA': 6562}
    mask_on_band = onBand_mask(d[onbandline],fname_on,cat) 
    mask_off_band = offBand_mask(d[onbandline],fname_off,cat)
    print(f"Number of galaxies with {onbandline} on F{fname_on} and off F{fname_off}: {np.sum(mask_on_band&mask_off_band)} vs input size of {len(cat)}")
    return mask_on_band&mask_off_band

now we divide sample into 4 z ranges. We can choose fitting filters to keep all targets!

In [ ]:
mul_Lcut = hst_sdss148
zranges = [0.1,0.14,0.2,0.3,0.4]
zcuts=[(mul_Lcut['Z']< zranges[i+1]) & (mul_Lcut['Z'] > zranges[i]) for i in range(len(zranges)-1)]

oiii_f547_621 = test_filter_set('547M','621M','OIII',mul_Lcut[zcuts[0]])
ha_f763_689 = test_filter_set('763M','689M','HA',mul_Lcut[zcuts[1]])
oiii_f621_689 = test_filter_set('621M','689M','OIII',mul_Lcut[zcuts[2]])
oiii_f689_763 = test_filter_set('689M','763M','OIII',mul_Lcut[zcuts[3]])

fig,ax = plt.subplots(figsize=(12,5))
# plot subsample by z-ranges
sample_colors = ['k','orangered','dodgerblue','darkorange', 'plum']
# choice for filling in OIII or HA hist
fills = [True, False, True, True]
[plt_subsample(mul_Lcut,ax,zranges[i],zranges[i+1], sample_colors[i],fills[i]) for i in range(len(zranges)-1)]
# plot filters
colors = ['g','orange','r','darkred','purple']
[plt_filter(list(flt.keys())[n],ax) for n in range(5)]


# kind of sanity checking that the filters work for each subsample
for m,zcut,wl in zip([oiii_f547_621,ha_f763_689,oiii_f621_689,oiii_f689_763],zcuts,[5007,6562,5007,5007]):
    # plot subsample by z-cut and on/off band filter mask
    plt.hist((1+mul_Lcut['Z'][m&zcut])*wl,histtype='step',color='k')

[ax.axvline(6548*i,c='k',alpha=0.5,label=l) for i,l in zip([1.13,1.26],['','NII'])]
ax.axvline(5007*1.25,c='b',alpha=0.5)

ax.set_xlabel("wavelength (angstroms)")
ax.set_ylabel("number of sources")
ax.set_title("shifted [OIII] and HA emission lines of 152 sample")
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.text(5400,10,"OIII5007")
ax.text(7300,10, "HA")
ax.set_xlim((4700,9500))
;

### search target and filter in HST
we need to see if hst has observed a target in the filter we want already. we found 4 objects observed and entries with 'target_name' "DARK' or 'DARK_NM'. Search the proposal id for the weird names, and those were probably calibration/maintainance stuffs

In [ ]:
from astroquery.esa.hubble import ESAHubble
def pos(row):
    """make skyCoord object for HST coord cone search"""
    return SkyCoord(ra=row['RA']*u.deg, dec=row['DEC']*u.deg)

def search_hst(mul171_,filters_=["F547M","F621M","F689M","F763M"]):
    """search if sample has been observed with HST in one of the filters in input list
        returns the search results, indices with search errors, and indices of objects that have been observed before"""
    res = []
    erred_ind = []
    esahubble = ESAHubble()
    for i in list(mul171_.index):
        try:# try to search in hubble given a mullaney df
            res.append(esahubble.cone_search_criteria(coordinates=pos(mul171_.loc[i]),
                                                radius=7*u.arcmin,
                                                instrument_name = ['WFC3'],
                                                filters = filters_))
        except: # if there is an error, save index so can find out why
            erred_ind.append(i)
    # index where there is a search match in HST
    observed_ind = [i for i in range(len(res)) if len(res[i])!=0]
    return res, erred_ind, observed_ind

In [ ]:
res, erred_ind, observed_ind = search_hst(hst_sdss148)
hstsearch = pd.concat([Table(r).to_pandas() for r in res])
hstsearch['DESIG'] = make_desig(hstsearch)

### binary calculation

we want to know how many binaries we expect to find from our SNAP given the GWB. From GWB, Kris gave a number density of binaries. We use GSMF and MBHMF from Liepold+2024 to calculate the number density of massive galaxies and massive BHs. The fraction is then the ratio of the 2

In [ ]:
from scipy.integrate import quad

def f(x,alpha,beta,phi):
    """integrand of SChecter function"""
    return phi*x**alpha*np.exp(-x**beta)  

def n_massive_dens(logMcut_,logMbhcut_):
    """calculate the number density of binary given mass cuts"""
    # GSMF constants
    phi1 = 10**-4.85
    phi2 = 10**-2.85
    Ms = 10**11.33
    alpha1 = 0.92
    alpha2 = -1.38
    # BHMF constants
    alpha_bh = -1.27
    beta_bh = 0.45
    phi_bh = 10**-2
    Msbh = 10**8.09
    # numerically integrate the number density of massive galaxies (note the GSMF is a sum of 2 PS functions)
    result1, _ = quad(f, 10**logMcut_/Ms, np.inf, args=(alpha1,1,phi1))
    result2, _ = quad(f, 10**logMcut_/Ms, np.inf, args=(alpha2,1,phi2)) 
    n_massive_gal = result1+result2
    # number density of massive BHs
    n_massive_bh, _ = quad(f, 10**logMbhcut_/Msbh, np.inf, args=(alpha_bh,beta_bh,phi_bh)) 
    return n_massive_gal, n_massive_bh

def n_bi(n_massive_,n_binary_):
    """calculate number of binary and galaxies given their number densities
        find dual fraction and expected observations"""
    # get binary number 
    # SDSS covers only part of sky
    f_sky = 9380/41253
    # comoving volume in z-range
    volume = (cosmo.comoving_volume(0.4)-cosmo.comoving_volume(0.1)).value
    # Number of binary
    N_c22 = n_binary_*volume*f_sky
    # get massive gal number
    N_massive_gal = n_massive_*volume*f_sky
    # dual fraction
    f_dual = N_c22/N_massive_gal
    # results
    print(f"{N_c22:.0f} binaries in SDSS in z=0.14-0.22")
    print(f"{N_massive_gal:.2e} massive galaxies")
    print(f"dual fraction:{f_dual*100:.2f}%")
    print(f"We can observe {f_dual*50:.0f}/50 binaries")

In [ ]:
# define cuts and binary density
logMcut = 11
logMbhcut = 8.32
n_bin = 2.3e-4
# calculate number of massive things
n_massive_gal, n_massive_bh = n_massive_dens(logMcut_=logMcut, logMbhcut_=logMbhcut)

# print results
print("GSMF")
n_bi(n_massive_gal,n_bin)
print("\nBHMF")
n_bi(n_massive_bh,n_bin)